### Statistics Module

https://docs.python.org/3/library/statistics.html#module-statistics

Let's start byt importing the `statistics` module, renaming it as `stats` to make my typing life easier :-)

In [1]:
import statistics as stats

#### Measures of Central Location

First let's look at the mean functions:

In [2]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [3]:
stats.mean(data)

5.5

We also have the `fmean` function, which will first convert all the elements to floats, and then calculate the mean:

In [4]:
stats.fmean(data)

5.5

The main difference is that all the elements are first converted to floats, and the algorithm is faster than using `mean`.

In [5]:
from time import perf_counter

In [6]:
start = perf_counter()
for _ in range(100_000):
    stats.mean(data)
end = perf_counter()
print(end - start)

0.8911100999976043


In [7]:
start = perf_counter()
for _ in range(100_000):
    stats.fmean(data)
end = perf_counter()
print(end - start)

0.051282899992656894


Why would ever use `mean` then?

Well consider this data, a list of integers, whose mean should actually be an integer:

In [8]:
data = [1, 2, 3, 4, 5]

In [9]:
stats.fmean(data)

3.0

In [10]:
stats.mean(data)

3

As you can see, the integer type was preserved with `mean`. But not always:

In [11]:
data = [1, 2, 3, 4, 5, 6]

In [12]:
stats.mean(data)

3.5

In [13]:
stats.fmean(data)

3.5

There are other means available in the statistics module (geometric, harmonic).

Also, we can calculate the median of a data set.

The median is not always a member of the data set - it may fall between two values in the data set.

In [14]:
data = [1, 1, 2, 3, 4, 5, 6, 6]

In [15]:
stats.median(data)

3.5

As we can see the median is between `3` and `4`, so `3.5` which is not in the set.

If you need an element from the data set, you may have to choose either the lower of the two values "around" the median, or the higher.

In [16]:
stats.median_low(data)

3

In [17]:
stats.median_high(data)

4

The `mode` can also be calculated. If more than one value qualifies as the mode, the first one in the list is returned.

For example, with `data`:

In [18]:
data

[1, 1, 2, 3, 4, 5, 6, 6]

we can see that we have two candidates for the mode: `1` and `6`.

In [19]:
stats.mode(data)

1

If we prefer, we can obtain all the modes, as a list:

In [20]:
stats.multimode(data)

[1, 6]

The `mode` function can also work with nominal data:

In [21]:
data = ['one', 'two', 'three', 'three', 'four', 'four']

In [22]:
stats.mode(data)

'three'

In [23]:
stats.multimode(data)

['three', 'four']

This `mode` function can be very useful for identifying elements that occur the most often in some dataset.

#### Measures of Spread

We also have standard deviation and variance functions - population and sample variants.

In [24]:
import random

In [25]:
random.seed(0)

data = [random.gauss(0, 2) for _ in range(10_000)]

In [26]:
stats.mean(data), stats.median(data)

(-0.0015217251679571582, 0.003173735624808653)

We can calculate the population std dev and variance:

In [27]:
stats.pstdev(data), stats.pvariance(data)

(1.9947180848823642, 3.978900238156767)

Or if `data` is a sample, we can calculate the sample std dev/variance:

In [28]:
stats.stdev(data), stats.variance(data)

(1.9948178282674247, 3.9792981679735644)

We can generate quantiles.

For example to generate quartiles (where data excludes extremes):

In [29]:
stats.quantiles(data, n=4)

[-1.3875586003005163, 0.003173735624808653, 1.3480210310232563]

And if the data includes the extremes:

In [30]:
stats.quantiles(data, n=4, method='inclusive')

[-1.3874592627504039, 0.003173735624808653, 1.3478465657948888]

#### Normal Distribution

The `statistics` module also includes a special data type (class), `NormalDist` that can be used to create and manipulate normal distributions.

In [31]:
d1 = stats.NormalDist(0, 1)

In [32]:
d1

NormalDist(mu=0.0, sigma=1.0)

This object has properties for mean, median, variance, std dev:

In [33]:
d1.mean, d1.median, d1.stdev, d1.variance

(0.0, 0.0, 1.0, 1.0)

There is also a PDF function, as well as a CDF function.

For example, the CDF function will return the probability that some random variable X <= x:

In [34]:
d1.cdf(0)

0.5

In [35]:
d1.cdf(3)

0.9986501019683699

These  objects also support translation and scaling:

In [36]:
d1 + 10

NormalDist(mu=10.0, sigma=1.0)

In [37]:
d1 * 2

NormalDist(mu=0.0, sigma=2.0)

In [38]:
d2 = 2 * d1 + 1

In [39]:
d2

NormalDist(mu=1.0, sigma=2.0)

We can measure the overlapping area between those two distributions:

In [40]:
d1.overlap(d2)

0.6099343398789443

In [41]:
d2.overlap(d1)

0.6099343398789443

We can also generate quantiles for the distribution:

In [42]:
d1.quantiles(4)

[-0.6744897501960817, 0.0, 0.6744897501960817]

And finally we can also combine two normal distributions, where the resulting mean will be the sum of the means, and the resulting variance will be the sum of the variances.

In [43]:
d1, d2

(NormalDist(mu=0.0, sigma=1.0), NormalDist(mu=1.0, sigma=2.0))

In [44]:
d1 + d2

NormalDist(mu=1.0, sigma=2.23606797749979)

In [45]:
d1.variance, d2.variance, (d1 + d2).variance

(1.0, 4.0, 5.000000000000001)

In [46]:
from math import sqrt

sqrt(d1.variance + d2.variance)

2.23606797749979

We can also sample data from the distribution:

In [47]:
random.seed(0)

d1.samples(n=10)

[0.9417154046806644,
 -1.3965781047011498,
 -0.6797144480784211,
 0.3705035674606598,
 -1.016348894188071,
 -0.07212002278507135,
 0.17919648727485687,
 -0.8310992152709882,
 -1.3090373644593587,
 0.1938877412491041]